In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold



In [52]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('F:/notebook_working/kaggle_compe/train.csv').fillna(' ')
test = pd.read_csv('F:/notebook_working/kaggle_compe/test.csv').fillna(' ')

#Insert one new row in train
train.loc[159571, 'id'] = "newnewnew"
train.loc[159571, 'comment_text'] = "This is just a new entry"
train.loc[159571, 'toxic'] = 0
train.loc[159571, 'severe_toxic'] = 0
train.loc[159571, 'obscene'] = 0
train.loc[159571, 'threat'] = 0
train.loc[159571, 'insult'] = 0
train.loc[159571, 'identity_hate'] = 0
#train.loc[159571,]


train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])



In [53]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    encoding='utf-8',
    lowercase=True,
    #min_df=0.00001,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2))
    #max_features=10000)




In [59]:
print len(train_text)
print len(test_text)

39893.0
153164


In [56]:
a= (len(train_text))/4
b= (len(test_text))/4
train_text1 = train_text[:a]
train_text2 = train_text[a:2*a]
train_text3 = train_text[2*a:3*a]
train_text4 = train_text[3*a:4*a]
test_text1 = test_text[:b]
test_text2 = test_text[b:2*b]
test_text3 = test_text[2*b:3*b]
test_text4 = test_text[3*b:4*b]

In [60]:

word_vectorizer.fit(train_text1)
train_word_features1 = word_vectorizer.transform(train_text1)
train_word_features1.shape

(39893, 909035)

In [73]:
#print train_word_features1

In [61]:
word_vectorizer.fit(train_text2)
train_word_features2 = word_vectorizer.transform(train_text2)
train_word_features2.shape

(39893, 915248)

In [62]:
word_vectorizer.fit(train_text3)
train_word_features3 = word_vectorizer.transform(train_text3)
train_word_features3.shape

(39893, 902896)

In [63]:
word_vectorizer.fit(train_text4)
train_word_features4 = word_vectorizer.transform(train_text4)
train_word_features4.shape

(39893, 906865)

In [64]:
word_vectorizer.fit(test_text1)
test_word_features1 = word_vectorizer.transform(test_text1)
test_word_features1.shape

(38291, 830037)

In [65]:
word_vectorizer.fit(test_text2)
test_word_features2 = word_vectorizer.transform(test_text2)
test_word_features2.shape

(38291, 838666)

In [66]:
word_vectorizer.fit(test_text3)
test_word_features3 = word_vectorizer.transform(test_text3)
test_word_features3.shape

(38291, 824649)

In [67]:
word_vectorizer.fit(test_text4)
test_word_features4 = word_vectorizer.transform(test_text4)
test_word_features4.shape

(38291, 842037)

In [14]:
#char_vectorizer.get_feature_names()

In [68]:
train_features = hstack([train_word_features1, train_word_features2, train_word_features3, train_word_features4])
test_features = hstack([test_word_features1, test_word_features2, test_word_features3, test_word_features4])


In [75]:
print type(train_features)


<class 'scipy.sparse.coo.coo_matrix'>


In [78]:


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train.loc[:a-1, class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_target, test_size=0.3, random_state=42)

    classifier = LogisticRegression()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

MemoryError: 

In [7]:
print train_word_features.shape
print train_char_features.shape
print test_word_features.shape
print test_char_features.shape

(159571, 318196)
(159571, 4116)
(153164, 318196)
(153164, 4116)


In [ ]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    #X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)

    skf=StratifiedKFold(n_splits=3, shuffle=False)
    #skf=StratifiedKFold(n_splits=3, shuffle=True)
    for train_indices, valid_indices in skf.split(train_word_features, train_target):
        X_train, y_train = train_word_features[train_indices], train_target[train_indices]
        X_valid, y_valid = train_word_features[valid_indices], train_target[valid_indices]
    
    classifier = LogisticRegression()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

In [13]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)

    classifier = SVC()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

CV score for class toxic is 0.953617374677
CV score for class severe_toxic is 0.878066506914
CV score for class obscene is 0.978812954813
CV score for class threat is 0.885887012223
CV score for class insult is 0.95942138282
CV score for class identity_hate is 0.871775257044
Total CV score is 0.921263414748


In [9]:
submission[class_name] = classifier.predict_proba(test_features)[:, 1]
submission.to_csv('submission.csv', index=False)